In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_excel("/content/drive/MyDrive/Internships/Prasunet Foundation-Data Science/dataset/cars.xlsx")

In [3]:
df.head()

,name_model,make_year,registration_year,fuel_type,km_driven,transmission,no_of_owner,insurance_validity,insurance_type,rto,...,tyre_life,avg_tyre_life%,mileage_kmpl,bootspace,seating_capacity,fuel_tank_capacity,displacement,total_cost,emi_per_month,interest_for_loan_duration
0,2015 Nissan Micra XV CVT Petrol,Jul 2015,Oct 2015,Petrol,41000,Automatic,2,2026-03-01,Third Party,KA04,...,Front\n \n L-70%\n R-90%\n \n Rear\n \n L-40%\...,67,15.0,NaN,Seating capacity\n \n 5 units,NaN,Displacement\n \n 1200 cc,362400.0,6040.0,96900.0
1,2021 Ford Freestyle Titanium Plus 1.2 Ti-VCT,Jun 2021,Jul 2021,Petrol,31000,Manual (Regular),1,2025-07-01,Comprehensive,KA51,...,Front\n \n L-45%\n R-35%\n \n Rear\n \n L-35%\...,40,18.5,Boot space\n \n 257 litres,Seating capacity\n \n 5 units,NaN,NaN,768768.0,11648.0,221568.0
2,2015 Hyundai Grand i10 Sportz 1.2 Kappa VTVT,Feb 2015,Mar 2015,Petrol,20000,Manual (Regular),1,2026-03-01,Third Party,KA51,...,Front\n \n L-50%\n R-50%\n \n Rear\n \n L-50%\...,52,18.9,Boot space\n \n 256 litres,Seating capacity\n \n 5 units,NaN,NaN,493620.0,8227.0,132020.0
3,2018 Maruti Suzuki Swift ZDI AMT,Jun 2018,Dec 2018,Diesel,86000,Automatic (AMT),1,2025-12-01,Comprehensive,KA01,...,Front\n \n L-70%\n R-60%\n \n Rear\n \n L-60%\...,65,25.2,Boot space\n \n 268 litres,Seating capacity\n \n 5 units,NaN,NaN,738408.0,11188.0,212808.0
4,2014 Maruti Suzuki Celerio VXI AMT,Oct 2014,Nov 2014,Petrol,55000,Automatic (AMT),1,2025-11-01,Comprehensive,KA04,...,Front\n \n L-75%\n R-75%\n \n Rear\n \n L-80%\...,66,23.1,Boot space\n \n 235 litres,Seating capacity\n \n 5 units,NaN,NaN,375984.0,7833.0,83984.0


In [4]:
def extract_brand_model(row):
    words = row.split()
    brand = words[1]

    # Define stop words that indicate variant/fuel/trim
    stop_words = ['Petrol', 'Diesel', 'AMT', 'CVT', 'TDI', 'AT', 'MT', 'Turbo', 'Hybrid', 'quattro']

    model_parts = []
    for word in words[2:]:
        if any(stop_word in word for stop_word in stop_words):
            break
        model_parts.append(word)

    model = ' '.join(model_parts).strip()
    return pd.Series([brand, model])

# Apply on your dataframe
df[['brand', 'model']] = df['name_model'].apply(extract_brand_model)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833 entries, 0 to 832
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   name_model                  833 non-null    object        
 1   make_year                   833 non-null    object        
 2   registration_year           833 non-null    object        
 3   fuel_type                   833 non-null    object        
 4   km_driven                   833 non-null    int64         
 5   transmission                833 non-null    object        
 6   no_of_owner                 833 non-null    int64         
 7   insurance_validity          833 non-null    datetime64[ns]
 8   insurance_type              833 non-null    object        
 9   rto                         833 non-null    object        
 10  car_location                833 non-null    object        
 11  tyre_life                   833 non-null    object        

In [6]:
# Clean 'bootspace', 'seating_capacity', and 'displacement'
for col in ["bootspace", "seating_capacity", "displacement", "fuel_tank_capacity"]:
    df[col] = df[col].astype(str).str.extract(r'(\d+)').astype(float)

In [7]:
df['bootspace']

,bootspace
0,NaN
1,257.0
2,256.0
3,268.0
4,235.0
...,...
828,490.0
829,490.0
830,490.0
831,251.0


In [8]:
df.tail()

,name_model,make_year,registration_year,fuel_type,km_driven,transmission,no_of_owner,insurance_validity,insurance_type,rto,...,mileage_kmpl,bootspace,seating_capacity,fuel_tank_capacity,displacement,total_cost,emi_per_month,interest_for_loan_duration,brand,model
828,2018 Nissan Sunny XE Diesel*,Apr 2017,Mar 2018,Diesel,91000,Manual (Regular),1,2026-03-01,Third Party,GJ38,...,22.7,490.0,5.0,41.0,1461.0,429858.0,6513.0,123858.0,Nissan,Sunny XE
829,2018 Nissan Sunny XE Diesel*,May 2017,Mar 2018,Diesel,104000,Manual (Regular),2,2026-03-01,Third Party,GJ06,...,22.7,490.0,5.0,41.0,1461.0,329802.0,4997.0,95052.0,Nissan,Sunny XE
830,2018 Nissan Sunny XL Diesel,Jan 2018,Jan 2018,Diesel,46000,Manual (Regular),2,2026-03-01,Third Party,GJ18,...,22.7,490.0,5.0,41.0,1461.0,455136.0,6896.0,131136.0,Nissan,Sunny XL
831,2015 Nissan Micra Active XV,May 2015,Oct 2015,Petrol,52000,Manual (Regular),1,2025-10-01,Comprehensive,GJ01,...,19.7,251.0,5.0,41.0,1198.0,302100.0,5035.0,80780.0,Nissan,Micra Active XV
832,2021 Nissan Magnite XV,Jan 2021,Jan 2021,Petrol,69000,Manual (Regular),1,2026-03-01,Third Party,GJ05,...,18.8,336.0,5.0,40.0,999.0,598092.0,9062.0,172353.0,Nissan,Magnite XV


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833 entries, 0 to 832
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   name_model                  833 non-null    object        
 1   make_year                   833 non-null    object        
 2   registration_year           833 non-null    object        
 3   fuel_type                   833 non-null    object        
 4   km_driven                   833 non-null    int64         
 5   transmission                833 non-null    object        
 6   no_of_owner                 833 non-null    int64         
 7   insurance_validity          833 non-null    datetime64[ns]
 8   insurance_type              833 non-null    object        
 9   rto                         833 non-null    object        
 10  car_location                833 non-null    object        
 11  tyre_life                   833 non-null    object        

In [10]:
has_duplicates = df.duplicated().any()
print("Duplicate rows exist?", has_duplicates)

Duplicate rows exist? True


In [11]:
total_duplicates = df.duplicated().sum()
print(f"Total duplicate rows: {total_duplicates}")

Total duplicate rows: 14


In [12]:
duplicate_rows = df[df.duplicated(keep=False)]  # `keep=False` marks ALL duplicates
print(duplicate_rows.sort_values(by=df.columns.tolist()))

                                            name_model make_year  \
15            2015 Audi A4 35 TDI Premium Plus Sunroof  Mar 2015   
105           2015 Audi A4 35 TDI Premium Plus Sunroof  Mar 2015   
9                          2016 Audi A4 35 TDI S Line*  Dec 2015   
103                        2016 Audi A4 35 TDI S Line*  Dec 2015   
22                          2018 Tata Tiago NRG Petrol  Aug 2018   
32                          2018 Tata Tiago NRG Petrol  Aug 2018   
53   2021 Jeep Compass 80 Anniversary 2.0 Diesel 4x...  Feb 2021   
100  2021 Jeep Compass 80 Anniversary 2.0 Diesel 4x...  Feb 2021   
290           2022 Hyundai i20 N Line N8 1.0 Turbo iMT  Jan 2022   
304           2022 Hyundai i20 N Line N8 1.0 Turbo iMT  Jan 2022   
20                             2022 Tata Altroz XT 1.2  Oct 2022   
31                             2022 Tata Altroz XT 1.2  Oct 2022   
26              2022 Tata Altroz XZ Plus 1.2 Dual Tone  Mar 2022   
29              2022 Tata Altroz XZ Plus 1.2 Dua

In [13]:
df = df.drop_duplicates()

In [14]:
df['total_cost'].head()

,total_cost
0,362400.0
1,768768.0
2,493620.0
3,738408.0
4,375984.0


In [15]:
df.describe()

,km_driven,no_of_owner,insurance_validity,avg_tyre_life%,mileage_kmpl,bootspace,seating_capacity,fuel_tank_capacity,displacement,total_cost,emi_per_month,interest_for_loan_duration
count,819.000000,819.000000,819,819.000000,819.000000,727.000000,795.000000,767.000000,738.000000,8.130000e+02,808.000000,8.080000e+02
mean,48587.301587,1.217338,2026-01-12 07:14:17.142857216,64.522589,18.862540,346.442916,5.096855,43.607562,1292.310298,8.959025e+05,14291.596535,2.559951e+05
min,1000.000000,1.000000,2024-12-01 00:00:00,24.000000,10.600000,84.000000,4.000000,15.000000,624.000000,1.371840e+05,3661.000000,9.184000e+03
25%,28000.000000,1.000000,2025-12-01 00:00:00,55.000000,16.900000,268.000000,5.000000,37.000000,999.000000,5.036460e+05,8453.500000,1.451460e+05
50%,42000.000000,1.000000,2026-03-01 00:00:00,65.000000,18.600000,345.000000,5.000000,43.000000,1198.000000,7.428960e+05,11801.000000,2.152230e+05
75%,63000.000000,1.000000,2026-03-01 00:00:00,74.000000,20.700000,413.500000,5.000000,50.000000,1497.000000,1.120548e+06,17068.000000,3.230218e+05
max,148000.000000,3.000000,2027-06-01 00:00:00,95.000000,34.000000,720.000000,7.000000,80.000000,2694.000000,4.256736e+06,64496.000000,1.226850e+06
std,28909.235501,0.465580,NaN,13.482413,2.998249,103.890089,0.480792,9.163932,312.504997,5.956321e+05,8742.895433,1.746929e+05


In [16]:
cols_to_fill = ['seating_capacity', 'bootspace', 'displacement', 'fuel_tank_capacity', 'total_cost']
for col in cols_to_fill:
    df[col] = (
        df.groupby('brand')[col]
        .apply(lambda group: group.fillna(group.mode().iloc[0]) if not group.mode().empty else group)
        .reset_index(level=0, drop=True)
    )

In [17]:
for col in ['bootspace', 'displacement']:
    if df[col].isna().sum() > 0:
        global_mode = df[col].mode().iloc[0]
        df[col] = df[col].fillna(global_mode)

In [18]:
df[cols_to_fill] = df[cols_to_fill].astype(int)

In [19]:
df['transmission'].unique()

array(['Automatic', 'Manual (Regular)', 'Automatic (AMT)',
       'Automatic (Torque Convertor)', 'Automatic (DCT)',
       'Automatic (CVT)', 'Automatic (DSG)', 'Manual (IMT)',
       'Manual (DCT)'], dtype=object)

In [20]:
df['transmission'].value_counts()

,count
transmission,
Manual (Regular),495
Automatic (Torque Convertor),72
Automatic (CVT),59
Automatic (AMT),58
Automatic,54
Automatic (DCT),37
Manual (IMT),29
Automatic (DSG),14
Manual (DCT),1


In [21]:
df['transmission'] = df['transmission'].apply(
    lambda x: 'Automatic' if 'Automatic' in str(x) else 'Manual')

In [22]:
df = df.drop('tyre_life', axis=1)
df = df.drop('rto', axis=1)
df = df.drop('emi_per_month', axis=1)
df = df.drop('interest_for_loan_duration', axis=1)

In [23]:
df['insurance_type'].unique()

array(['Third Party', 'Comprehensive', 'Zero Dep (Comprehensive)'],
      dtype=object)

In [24]:
df['insurance_type'].value_counts()

,count
insurance_type,
Third Party,577
Comprehensive,240
Zero Dep (Comprehensive),2


In [25]:
df['insurance_type'] = df['insurance_type'].replace('Zero Dep (Comprehensive)', 'Comprehensive')

In [26]:
df['insurance_type'].value_counts()

,count
insurance_type,
Third Party,577
Comprehensive,242


In [27]:

df['fuel_type'].value_counts()

,count
fuel_type,
Petrol,684
Diesel,133
Petrol,1
hybrid,1


In [28]:
df['fuel_type'] = df['fuel_type'].apply(lambda x: 'Petrol' if 'petrol' in str(x).lower() else 'Diesel')

In [29]:
df['fuel_type'].value_counts()

,count
fuel_type,
Petrol,685
Diesel,134


In [30]:
df['car_location'].unique()

array(['Bangalore', 'Delhi', 'Hyderabad', 'Mumbai', 'Chennai', 'Kolkata',
       'Ahmedabad'], dtype=object)

In [31]:
df['car_location'].value_counts()

,count
car_location,
Ahmedabad,134
Bangalore,130
Chennai,126
Mumbai,117
Delhi,108
Kolkata,106
Hyderabad,98


In [32]:
df['brand'].unique()

array(['Nissan', 'Ford', 'Hyundai', 'Maruti', 'MG', 'Honda', 'Audi',
       'Mercedes-Benz', 'Land', 'Tata', 'Volkswagen', 'Renault', 'Toyota',
       'Mahindra', 'BMW', 'Jeep', 'Skoda', 'Kia', 'Datsun', 'Volvo',
       'Chevrolet', 'Jaguar'], dtype=object)

In [33]:
df['brand'].value_counts()

,count
brand,
Maruti,119
Hyundai,103
Renault,66
Tata,61
Honda,57
Ford,56
Mahindra,56
Volkswagen,55
Kia,55


## Manual Mapping - Label Encoding

In [34]:
insurance_type_map = {
    'Comprehensive': 1,
    'Third Party': 0
}
df['insurance_type_encoded'] = df['insurance_type'].map(insurance_type_map)

In [35]:
fuel_type_map = {
    'Petrol': 0,
    'Diesel': 1
}
df['fuel_type_encoded'] = df['fuel_type'].map(fuel_type_map)

In [36]:
transmission_map = {
    'Manual': 0,
    'Automatic': 1
}
df['transmission_encoded'] = df['transmission'].map(transmission_map)

In [37]:
city_map = {
    'Bangalore': 0,
    'Mumbai': 1,
    'Delhi': 2,
    'Hyderabad': 3,
    'Chennai': 4,
    'Ahmedabad': 5,
    'Kolkata': 6
}
df['car_location_encoded'] = df['car_location'].map(city_map)

## Feature Engineering

In [38]:
# Extract only the year (if month + year or messy formats exist)
df['make_year'] = df['make_year'].str.extract(r'(\d{4})')
df['registration_year'] = df['registration_year'].str.extract(r'(\d{4})')

# Convert to numeric (int) so you can use them directly for calculations
df['make_year'] = pd.to_numeric(df['make_year'], errors='coerce')
df['registration_year'] = pd.to_numeric(df['registration_year'], errors='coerce')

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 819 entries, 0 to 832
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   name_model              819 non-null    object        
 1   make_year               819 non-null    int64         
 2   registration_year       819 non-null    int64         
 3   fuel_type               819 non-null    object        
 4   km_driven               819 non-null    int64         
 5   transmission            819 non-null    object        
 6   no_of_owner             819 non-null    int64         
 7   insurance_validity      819 non-null    datetime64[ns]
 8   insurance_type          819 non-null    object        
 9   car_location            819 non-null    object        
 10  avg_tyre_life%          819 non-null    int64         
 11  mileage_kmpl            819 non-null    float64       
 12  bootspace               819 non-null    int64         


In [40]:
from datetime import datetime
current_year = datetime.now().year

df['car_age'] = current_year - df['registration_year']
df['used_years'] = current_year - df['make_year']

In [41]:
# Calculate remaining insurance in months
today = pd.to_datetime(datetime.today())
df['insurance_remaining_months'] = ((df['insurance_validity'] - today).dt.days // 30).clip(lower=0)

In [42]:
# Mileage per cc (efficiency)
df['mileage_per_cc'] = df['mileage_kmpl'] / df['displacement']

# Boot space per seat (for comfort/load capacity)
df['bootspace_per_seat'] = df['bootspace'] / df['seating_capacity']

# Tyre Wear Score (bucketed)
def categorize_tyre_life(val):
    if val >= 85:
        return 'Excellent'
    elif val >= 60:
        return 'Good'
    elif val >= 35:
        return 'Moderate'
    else:
        return 'Poor'

df['tyre_life_category'] = df['avg_tyre_life%'].apply(categorize_tyre_life)

# Fuel Efficiency Category (ARAI Mileage buckets)
def fuel_efficiency_group(mileage):
    if mileage >= 25:
        return 'Ultra Efficient'
    elif mileage >= 20:
        return 'Efficient'
    elif mileage >= 15:
        return 'Average'
    else:
        return 'Low Efficiency'

df['mileage_category'] = df['mileage_kmpl'].apply(fuel_efficiency_group)

In [43]:
# Tyre Life Category (ordinal)
tyre_map = {'Poor': 0, 'Moderate': 1, 'Good': 2, 'Excellent': 3}
df['tyre_life_encoded'] = df['tyre_life_category'].map(tyre_map)

# Fuel Efficiency Category (ordinal)
fuel_map = {'Low Efficiency': 0, 'Average': 1, 'Efficient': 2, 'Ultra Efficient': 3}
df['fuel_efficiency_encoded'] = df['mileage_category'].map(fuel_map)

In [44]:
def remove_price_outliers(df, price_col='total_cost'):
    Q1 = df[price_col].quantile(0.25)
    Q3 = df[price_col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = max(Q1 - 1.5 * IQR, 10000)  # Enforce minimum price of ₹10,000
    upper_bound = Q3 + 1.5 * IQR
    df_clean = df[(df[price_col] >= lower_bound) & (df[price_col] <= upper_bound)]
    print(f"Removed {len(df) - len(df_clean)} outliers (out of {len(df)})")
    return df_clean

In [45]:
df = remove_price_outliers(df, price_col='total_cost')

Removed 38 outliers (out of 819)


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 781 entries, 0 to 832
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   name_model                  781 non-null    object        
 1   make_year                   781 non-null    int64         
 2   registration_year           781 non-null    int64         
 3   fuel_type                   781 non-null    object        
 4   km_driven                   781 non-null    int64         
 5   transmission                781 non-null    object        
 6   no_of_owner                 781 non-null    int64         
 7   insurance_validity          781 non-null    datetime64[ns]
 8   insurance_type              781 non-null    object        
 9   car_location                781 non-null    object        
 10  avg_tyre_life%              781 non-null    int64         
 11  mileage_kmpl                781 non-null    float64       
 12 

In [47]:
df.head()

,name_model,make_year,registration_year,fuel_type,km_driven,transmission,no_of_owner,insurance_validity,insurance_type,car_location,...,car_location_encoded,car_age,used_years,insurance_remaining_months,mileage_per_cc,bootspace_per_seat,tyre_life_category,mileage_category,tyre_life_encoded,fuel_efficiency_encoded
0,2015 Nissan Micra XV CVT Petrol,2015,2015,Petrol,41000,Automatic,2,2026-03-01,Third Party,Bangalore,...,0,10,10,10,0.012500,67.2,Good,Average,2,1
1,2021 Ford Freestyle Titanium Plus 1.2 Ti-VCT,2021,2021,Petrol,31000,Manual,1,2025-07-01,Comprehensive,Bangalore,...,0,4,4,2,0.012342,51.4,Moderate,Average,1,1
2,2015 Hyundai Grand i10 Sportz 1.2 Kappa VTVT,2015,2015,Petrol,20000,Manual,1,2026-03-01,Third Party,Bangalore,...,0,10,10,10,0.015789,51.2,Moderate,Average,1,1
3,2018 Maruti Suzuki Swift ZDI AMT,2018,2018,Diesel,86000,Automatic,1,2025-12-01,Comprehensive,Bangalore,...,0,7,7,7,0.021053,53.6,Good,Ultra Efficient,2,3
4,2014 Maruti Suzuki Celerio VXI AMT,2014,2014,Petrol,55000,Automatic,1,2025-11-01,Comprehensive,Bangalore,...,0,11,11,6,0.019298,47.0,Good,Efficient,2,2


In [48]:
df['model']

,model
0,Micra XV
1,Freestyle Titanium Plus 1.2 Ti-VCT
2,Grand i10 Sportz 1.2 Kappa VTVT
3,Suzuki Swift ZDI
4,Suzuki Celerio VXI
...,...
828,Sunny XE
829,Sunny XE
830,Sunny XL
831,Micra Active XV


In [49]:
df.to_excel("/content/drive/MyDrive/Internships/Prasunet Foundation-Data Science/dataset/cleaned_car_data.xlsx", index=False)